In [ ]:
%run common_setting.ipynb


import numpy as np
import pandas as pd
import re
import statistics
import math
import pickle



<Preset> includes:
"dataset_atom_init": C:\Users\NDO00\.xenonpy\dataset\atom_init.pd.xz
"dataset_elements": C:\Users\NDO00\.xenonpy\dataset\elements.pd.xz
"dataset_elements_completed": C:\Users\NDO00\.xenonpy\dataset\elements_completed.pd.xz
"mp_samples": C:\Users\NDO00\.xenonpy\userdata\mp_samples.pd.xz

In [ ]:
#元素ごとのパラメーターシートの読み込み(xenonpyがインストールされている場合のみ有効)
from xenonpy.datatools import preset
elems = preset.dataset_elements  # raw data
elems_completed = preset.dataset_elements_completed  # completed data
preset

In [ ]:
#xenonpyをインストールしていない場合は、このセルを実行して元素の各種パラメータを読み込む。
elems=pd.read_csv("elems.csv",index_col=0)
elems.head()

In [ ]:
from __future__ import annotations
from typing import TypeVar, List, Dict
import random
from random import choices, randrange, shuffle
#from random import randomすると、random.sample()とできなくなる。
from heapq import nlargest
from abc import ABC, abstractmethod
from copy import deepcopy
from datetime import datetime

## 遺伝的アルゴリズムのコード

### 抽象クラスの定義(複数の題材の問題を解く際に必要。今回は遺伝的アルゴリズムで１個の問題しか解かないため不要かも)

In [ ]:
class Chromosome(ABC):
    """
    染色体（遺伝的アルゴリズムの要素1つ分）を扱う抽象クラス。
    """

    @abstractmethod
    def fitness(self) -> float:
        """
        対象の問題に対する染色体の優秀さを取得する評価関数Y用の
        抽象メソッド。

        Returns
        -------
        fitness : float
            対象の問題に対する染色体の優秀さの値。高いほど問題に
            適した染色体となる。
            遺伝的アルゴリズムの終了判定などにも使用される。
        """
        ...

    @classmethod
    @abstractmethod
    def random_instance(cls) -> Chromosome:
        """
        ランダムな特徴（属性値）を持ったインスタンスを生成する
        抽象メソッド。

        Returns
        -------
        instance : Chromosome
            生成されたインスタンス。
        """
        ...

    @abstractmethod
    def mutate(self) -> None:
        """
        染色体を（突然）変異させる処理の抽象メソッド。
        インスタンスの属性などのランダムな別値の設定などが実行される。
        """
        ...

    @abstractmethod
    def crossover(self, other: Chromosome) -> List[Chromosome]:
        """
        引数に指定された別の個体を参照し交叉を実行する。

        Parameters
        ----------
        other : Chromosome
            交叉で利用する別の個体。

        Returns
        -------
        result_chromosomes : list of Chromosome
            交叉実行後に生成された2つの個体（染色体）。
        """
        ...

    def __lt__(self, other: Chromosome) -> bool:
        """
        個体間の比較で利用する、評価関数の値の小なり比較用の関数。

        Parameters
        ----------
        other : Chromosome
            比較対象の他の個体。

        Returns
        -------
        result_bool : bool
            小なり条件を満たすかどうかの真偽値。
        """
        return self.fitness() < other.fitness()

In [ ]:
C = TypeVar('C', bound=Chromosome)


class GeneticAlgorithm:

    SelectionType = int
    SELECTION_TYPE_ROULETTE_WHEEL: SelectionType = 1
    SELECTION_TYPE_TOURNAMENT: SelectionType = 2

    def __init__(
            self, initial_population: List[C],
            threshold: float,
            max_generations: int, mutation_probability: float,
            crossover_probability: float,
            selection_type: SelectionType) -> None:
        """
        遺伝的アルゴリズムを扱うクラス。

        Parameters
        ----------
        initial_population : list of Chromosome
            最初の世代の個体群（染色体群）。
        threshold : float
            問題解決の判定で利用するしきい値。この値を超える個体が
            発生した時点で計算が終了する。
        max_generations : int
            アルゴリズムで実行する最大世代数。
        mutation_probability : float
            変異確率（0.0～1.0）。
        crossover_probability : float
            交叉確率（0.0～1.0）。
        selection_type : int
            選択方式。以下のいずれかの定数値を指定する。
            - SELECTION_TYPE_ROULETTE_WHEEL
            - SELECTION_TYPE_TOURNAMENT
        """
        self._population: List[Chromosome] = initial_population
        self._threshold: float = threshold
        self._max_generations: int = max_generations
        self._mutation_probability: float = mutation_probability
        self._crossover_probability: float = crossover_probability
        self._selection_type: int = selection_type

    def _exec_roulette_wheel_selection(self) -> List[Chromosome]:
        """
        ルーレット選択を行い、交叉などで利用する2つの個体（染色体）を
        取得する。

        Returns
        -------
        selected_chromosomes : list of Chromosome
            選択された2つの個体（染色体）を格納したリスト。選択処理は評価関数
            （fitnessメソッド）による重みが設定された状態でランダムに抽出される。

        Notes
        -----
        評価関数の結果の値が負になる問題には利用できない。
        """
        weights: List[float] = [
            chromosome.fitness() for chromosome in self._population]
        selected_chromosomes: List[Chromosome] = choices(
            self._population, weights=weights, k=2)
        return selected_chromosomes

    def _exec_tournament_selection(self) -> List[Chromosome]:
        """
        トーナメント選択を行い、交叉などで利用するための2つの個体
        （染色体）を取得する。

        Returns
        -------
        selected_chromosomes : list of Chromosome
            選択された2つの個体（染色体）を格納したリスト。トーナメント
            用に引数で指定された件数分抽出された中から上位の2つの個体が
            設定される。
        """
        participants_num: int = len(self._population) // 2
        participants: List[Chromosome] = choices(self._population, k=participants_num)
        selected_chromosomes: List[Chromosome] = nlargest(n=2, iterable=participants)
        return selected_chromosomes

    def _to_next_generation(self) -> None:
        """
        次世代の個体（染色体）を生成し、個体群の属性値を生成した
        次世代の個体群で置換する。
        """
        new_population: List[Chromosome] = []

        # 元の個体群の件数が奇数件数の場合を加味して件数の比較は等値ではなく
        # 小なりの条件で判定する。
        while len(new_population) < len(self._population):
            parents: List[Chromosome] = self._get_parents_by_selection_type()
            next_generation_chromosomes: List[Chromosome] = \
                self._get_next_generation_chromosomes(parents=parents)
            new_population.extend(next_generation_chromosomes)

        # 2件ずつ次世代のリストを増やしていく都合、元のリストよりも件数が
        # 多い場合は1件リストから取り除いてリストの件数を元のリストと一致させる。
        if len(new_population) > len(self._population):
            del new_population[0]

        self._population = new_population

    def _get_next_generation_chromosomes(
            self, parents: List[Chromosome]) -> List[Chromosome]:
        """
        算出された親の2つの個体のリストから、次世代として扱う
        2つの個体群のリストを取得する。
        一定確率で交叉や変異させ、確率を満たさない場合には引数の値が
        そのまま次世代として設定される。

        Parameters
        ----------
        parents : list of Chromosome
            算出された親の2つの個体のリスト

        Returns
        -------
        next_generation_chromosomes : list of Chromosome
            次世代として設定される、2つの個体を格納したリスト。
        """
        random_val: float = random.random()
        next_generation_chromosomes: List[Chromosome] = parents
        if random_val < self._crossover_probability:
            next_generation_chromosomes = parents[0].crossover(
                other=parents[1])

        random_val = random.random()
        if random_val < self._mutation_probability:
            for chromosome in next_generation_chromosomes:
                chromosome.mutate()
        return next_generation_chromosomes

    def _get_parents_by_selection_type(self) -> List[Chromosome]:
        """
        選択方式に応じた親の2つの個体（染色体）のリストを取得する。

        Returns
        -------
        parents : list of Chromosome
            取得された親の2つの個体（染色体）のリスト。

        Raises
        ------
        ValueError
            対応していない選択方式が指定された場合。
        """
        if self._selection_type == self.SELECTION_TYPE_ROULETTE_WHEEL:
            parents: List[Chromosome] = self._exec_roulette_wheel_selection()
        elif self._selection_type == self.SELECTION_TYPE_TOURNAMENT:
            parents = self._exec_tournament_selection()
        else:
            raise ValueError(
                '対応していない選択方式が指定されています : %s'
                % self._selection_type)
        return parents

    def run_algorithm(self) -> Chromosome:
        """
        遺伝的アルゴリズムを実行し、実行結果の個体（染色体）のインスタンス
        を取得する。

        Returns
        -------
        betst_chromosome : Chromosome
            アルゴリズム実行結果の個体。評価関数でしきい値を超えた個体
            もしくはしきい値を超えない場合は指定された世代数に達した
            時点で一番評価関数の値が高い個体が設定される。
        """
        best_chromosome: Chromosome = \
            deepcopy(self._get_best_chromosome_from_population())
        for generation_idx in range(self._max_generations):
            print(

                f'世代数 : {generation_idx}'
                f'　最良個体 : {best_chromosome}'
            )

            if best_chromosome.fitness() >= self._threshold:
                return best_chromosome

            self._to_next_generation()

            currrent_generation_best_chromosome: Chromosome = \
                self._get_best_chromosome_from_population()
            current_gen_best_fitness: float = \
                currrent_generation_best_chromosome.fitness()
            if best_chromosome.fitness() < current_gen_best_fitness:
                best_chromosome = deepcopy(currrent_generation_best_chromosome)
        return best_chromosome

    def _get_best_chromosome_from_population(self) -> Chromosome:
        """
        個体群のリストから、評価関数の値が一番高い個体（染色体）を
        取得する。

        Returns
        -------
        best_chromosome : Chromosome
            リスト内の評価関数の値が一番高い個体。
        """
        best_chromosome: Chromosome = self._population[0]
        for chromosome in self._population:
            if best_chromosome.fitness() < chromosome.fitness():
                best_chromosome = chromosome
        return best_chromosome

### 遺伝的アルゴリズムのクラスの定義

## 合金の分析データシートの読み込み等

In [ ]:
#回帰係数や、各列の平均と標準偏差
df_gen=pd.read_csv("df_gen.csv",index_col=0)
display(df_gen)

#回帰直線の切片
with open('lasso_intercept.txt') as f:
    intercept = f.read()
intercept=float(intercept)
print(intercept)

,mean_MT,RADIUS_ASYM,mean_YM,YM_ASYM,delta_S,VEC,delta_H,TdeltaS/deltaH
mean,1836.935739,0.055290,181.635197,0.370857,0.013864,7.455948,-19.614809,3.272061
std,345.532893,0.015387,33.109600,0.107656,0.001109,1.342900,13.809095,8.067333
coef,-0.000000,38.822635,4.706428,56.367571,0.000000,-17.045764,-37.024298,-0.000000


515.0817899607567


In [ ]:
#エンタルピー表の読み込み
df_enthalpy_all=pd.read_csv("df_enthalpy_all.csv",index_col=0)
df_enthalpy_all.head()

,Li,B,C,Mg,Al,Si,Ca,Sc,Ti,V,...,Nb,Mo,Pd,Ag,Sn,Nd,Hf,Ta,W,Re
Li,0.0,-6.0,-61.0,0.0,-4.0,-30.0,-1.0,12.0,34.0,37.0,...,51.0,49,-40,-16,-18.0,7.0,30.0,48.0,50.0,29.0
B,-6.0,0.0,-10.0,-4.0,0.0,-14.0,-22.0,-55.0,-58.0,-42.0,...,-54.0,-34,-24,5,18.0,-49.0,-66.0,-54.0,-31.0,-25.0
C,-61.0,-10.0,0.0,-55.0,-36.0,-39.0,-89.0,-118.0,-109.0,-82.0,...,-102.0,-67,-32,-32,-23.0,-116.0,-123.0,-101.0,-60.0,-42.0
Mg,0.0,-4.0,-55.0,0.0,-2.0,-26.0,-6.0,-3.0,16.0,23.0,...,32.0,36,-40,-10,-9.0,-6.0,10.0,30.0,38.0,21.0
Al,-4.0,0.0,-36.0,-2.0,0.0,-19.0,-20.0,-38.0,-30.0,-16.0,...,-18.0,-5,-46,-4,4.0,-38.0,-39.0,-19.0,-2.0,-9.0


In [ ]:
#使用する元素の読み込み
def pickle_load(path):
    with open(path,mode='rb') as f:
        data = pickle.load(f)
        return data

In [ ]:
#使用する元素リストの命名
all_element_list=pickle_load("allelement.txt")
print(type(all_element_list))

#組成の候補(以後使用しないが、組成の割り振り方のイメージを把握するため書いた。1~20までの組成を１つ刻みに振る。
num=list(range(1, 21))
print(num)

<class 'list'>
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


# 合金用遺伝的アルゴリズムのクラス

In [ ]:
class Analysis_Alloy(Chromosome):


    #初期値としての合金の組成を入力する。
    def __init__(self, formula:str, kinds:int) -> None:

        self.formula=formula
        self.kinds=kinds

    #構成元素をリスト化する関数
    def elementlist(self) -> List:

        s=self.formula
        ele = re.findall("[a-zA-Z]+",s)
        return ele

    #組成比を返す関数
    def compositionlist(self)-> List:

        s=self.formula
        comp=re.findall("\d+(?:\.\d+)?",s)

        comp_flo = map(float, comp)
        comp_flo=list(comp_flo)

        a=sum(comp_flo)
        norm_comp_flo = [round(f/a,3) for f in comp_flo]
        return norm_comp_flo

    #融点を計算する関数
    def mean_melting_temperature(self):

        melting_temp_list=[]
        t=0
        s=self.formula
        for i in range(len(self.elementlist())):
            melting_temp_list.append(elems.loc[self.elementlist()[i]]["melting_point"])


        for i in range(len(self.elementlist())):
            t+=self.compositionlist()[i]*melting_temp_list[i]

        return t

    #半径の非対称性を計算する関数
    def radius_asymmetry(self):

        radius_list=[]
        D=0
        s=self.formula
        for i in range(len(self.elementlist())):
            radius_list.append(elems.loc[self.elementlist()[i]]["atomic_radius"])
        r=statistics.mean(radius_list)

        for i in range(len(self.elementlist())):
            D+=self.compositionlist()[i]*((1-(radius_list[i]/r))**2)

        K=math.sqrt(D)

        return K

    #ヤング率の平均を計算する関数
    def mean_youngs_modulus(self):

        youngs_modulus_list=[]
        D=0
        s=self.formula

        for i in range(len(self.elementlist())):
            youngs_modulus_list.append(elems.loc[self.elementlist()[i]]["youngs_modulus"])


        for i in range(len(self.elementlist())):
            D+=self.compositionlist()[i]*youngs_modulus_list[i]

        return D

    #ヤング率の非対称性を計算する関数
    def youngs_modulus_asymmetry(self):

        s=self.formula
        youngs_modulus_list=[]
        D=0

        for i in range(len(self.elementlist())):
            youngs_modulus_list.append(elems.loc[self.elementlist()[i]]["youngs_modulus"])
        e=statistics.mean(youngs_modulus_list)


        for i in range(len(self.elementlist())):
            D+=self.compositionlist()[i]*((1-(youngs_modulus_list[i]/e))**2)

        K=math.sqrt(D)

        return K

    #混合エントロピーを算出する関数
    def ideal_mixing_entropy(self):

        entropy_list=[]
        D=0
        s=self.formula
        for i in range(len(self.elementlist())):
            m=self.compositionlist()[i]*np.log(self.compositionlist()[i])
            m=m*(-8.314/1000)
            #もしRの何倍かしりたければ、この関数の返り値をRでわること
            entropy_list.append(m)
        e=sum(entropy_list)

        return e

    #価電子密度を算出する関数
    def valence_electron_concentration(self):

        valence_electron_list=[]
        VEC=0
        s=self.formula
        for i in range(len(self.elementlist())):
            vs=elems.loc[self.elementlist()[i]]["num_s_valence"]
            vp=elems.loc[self.elementlist()[i]]["num_p_valence"]
            vd=elems.loc[self.elementlist()[i]]["num_d_valence"]
            vf=elems.loc[self.elementlist()[i]]["num_f_valence"]

            valence_electron_list.append(vs+vp+vd+vf)

        for i in range(len(self.elementlist())):
            VEC+=self.compositionlist()[i]*valence_electron_list[i]

        return VEC

    #混合エンタルピーを算出する関数
    def mixing_enthalpy(self):

        H=0
        s=self.formula
        for i in range(len(self.elementlist())):
            for j in range(len(self.elementlist())):

                H+= self.compositionlist()[i]*self.compositionlist()[j]*df_enthalpy_all.loc[self.elementlist()[i],self.elementlist()[j]]

        return 4*H

    #Ωパラメーターの計算
    def S_over_H(self):

        s=self.formula

        omega=self.ideal_mixing_entropy()*self.mean_melting_temperature()/abs(self.mixing_enthalpy())

        if omega>100 or np.isnan(omega):
            omega=100

        return omega


    #単相での合成が可能か否かを返す関数。Ω-δのcritetion
    #条件１：エントロピー項がエンタルピーより大きいこと(エネルギー的に有利)
    #条件２：構成元素の原子半径の非対称性が小さい(同じ大きさ同士ほど合金を作りやすい)
    def criterion(self):
        if self.S_over_H()>=1.1 and self.radius_asymmetry()<=0.066:

            return True
        else:
            return False

    #適合度関数
    def fitness(self):

        hardness=intercept
        list=[]

        list.append(self.mean_melting_temperature())
        list.append(self.radius_asymmetry())
        list.append(self.mean_youngs_modulus())
        list.append(self.youngs_modulus_asymmetry())
        list.append(self.ideal_mixing_entropy())
        list.append(self.valence_electron_concentration())
        list.append(self.mixing_enthalpy())
        list.append(self.S_over_H())

        for i in range(len(list)):
            A=df_gen.loc["mean", df_gen.columns[i] ]
            B=df_gen.loc["std", df_gen.columns[i] ]
            C=df_gen.loc["coef", df_gen.columns[i] ]

            hardness+= C*(list[i]-A)/B

        return hardness

    #ランダムに初期構造を出す
    def random_instance(self) -> Analysis_alloy:

        #ランダムに元素を選び出す
        e=random.sample(all_element_list, self.kinds)
        #ランダムに組成を振る
        com=[random.randint(1, 20) for i in range(self.kinds)]
        new_formula=""
        for i in range(self.kinds):
            new_formula+=e[i]+str(com[i])+" "

        new_formula=new_formula[:-1]
        self.formula=new_formula

        #できた組成式の合成可能な基準をみたしていなければやり直し
        while self.criterion()==False :
            e=random.sample(all_element_list,self.kinds)
            com=[random.randint(1, 20) for i in range(self.kinds)]
            new_formula=""
            for i in range(self.kinds):
                new_formula+=e[i]+str(com[i])+" "

            new_formula=new_formula[:-1]
            self.formula=new_formula

        return Analysis_Alloy(new_formula,self.kinds)

    #交雑
    def crossover(self,other:Analysis_Alloy)->Tuple[Analysis_Alloy,Analysis_Alloy]:

        child1:Analysis_Alloy = deepcopy(self)
        child2:Analysis_Alloy = deepcopy(other)

        ele1=self.elementlist()
        com1=self.compositionlist()

        ele2=other.elementlist()
        com2=other.compositionlist()

        #２つ組成式の数字の部分だけを互いに入れ替え、返す。
        new_formula1=""
        new_formula2=""
        for i in range(self.kinds):
            new_formula1+=ele1[i]+str(com2[i])+" "

        for i in range(self.kinds):
            new_formula2+=ele2[i]+str(com1[i])+" "

        #最後の空白文字は取得しない
        new_formula1=new_formula1[:-1]
        new_formula2=new_formula2[:-1]

        child1.formula=new_formula1
        child1.formula=new_formula2

        return child1, child2

    #突然変異 この場合も初期値生成と同じく、ランダムな組成式の物質に変化させる。
    def mutate(self)-> None:
            #ランダムに元素を選び出す
        e=random.sample(all_element_list,self.kinds)
        #ランダムに組成を振る
        com=[random.randint(1, 20) for i in range(self.kinds)]
        new_formula=""
        for i in range(self.kinds):
            new_formula+=e[i]+str(com[i])+" "

        new_formula=new_formula[:-1]
        self.formula=new_formula
        #できた組成式が、合成可能な基準をみたしていなければやり直し
        while self.criterion()==False :
            e=random.sample(all_element_list,self.kinds)
            com=[random.randint(1, 20) for i in range(self.kinds)]
            new_formula=""
            for i in range(self.kinds):
                new_formula+=e[i]+str(com[i])+" "

            new_formula=new_formula[:-1]
            self.formula=new_formula

    #結果表示
    def __str__(self)-> str:

        fm=self.formula
        hn=self.fitness()

        return '組成式:{0} and 硬さ:{1}'.format(fm,round(hn,2))


## コードの実行

In [ ]:
if __name__ == '__main__':

    simple_equation_initial_population: List[Analysis_Alloy] = \
        [Analysis_Alloy("Cu1",5).random_instance() for _ in range(30)]
    ga: GeneticAlgorithm = GeneticAlgorithm(
        initial_population=simple_equation_initial_population,
        threshold=1000,
        max_generations=100,
        mutation_probability=0.2,
        crossover_probability=0.3,
        selection_type=GeneticAlgorithm.SELECTION_TYPE_TOURNAMENT)
    _ = ga.run_algorithm()

世代数 : 0　最良個体情報 : 組成式:W19 Ti8 Sn5 Sc3 Ta2 and 硬さ:775.05
世代数 : 1　最良個体情報 : 組成式:Ti0.118 W0.559 Hf0.029 Sn0.059 Ta0.235 and 硬さ:776.74
世代数 : 2　最良個体情報 : 組成式:Ti0.118 W0.559 Hf0.029 Sn0.059 Ta0.235 and 硬さ:776.74
世代数 : 3　最良個体情報 : 組成式:Ti0.118 W0.559 Hf0.029 Sn0.059 Ta0.235 and 硬さ:776.74
世代数 : 4　最良個体情報 : 組成式:Ti0.118 W0.559 Hf0.029 Sn0.059 Ta0.235 and 硬さ:776.74
世代数 : 5　最良個体情報 : 組成式:Ti0.118 W0.559 Hf0.029 Sn0.059 Ta0.235 and 硬さ:776.74
世代数 : 6　最良個体情報 : 組成式:Ti0.118 W0.559 Hf0.029 Sn0.059 Ta0.235 and 硬さ:776.74
世代数 : 7　最良個体情報 : 組成式:Ti0.118 W0.559 Hf0.029 Sn0.059 Ta0.235 and 硬さ:776.74


KeyboardInterrupt: 